# Machine Learning with Spark (15 points)

**Read through the entire workbook before beginning work to familiarize with what is being asked.**

In this exericise, you will work with a fairly large dataset and do some modeling and predictions using Spark Machine Learning. The dataset being used is the 2013 NYC Taxi Dataset [https://archive.org/details/nycTaxiTripData2013](https://archive.org/details/nycTaxiTripData2013). The original dataset consists of two files, `trip_data` and `trip_fare`, which need to be merged to create the dataset for modeling. 

The merge has already been performed and stored as a parquet file on S3 because there were some data issues that were not covered in class. The workbook [merge-fare-and-trip-data-create-parquet.ipynb](merge-fare-and-trip-data-create-parquet.ipynb) contains code for creating the merged set (for reference.)

This is an interactive PySpark session. Remember that when you open this notebook the SparkContext and SparkSession are already created, and they are in the sc and spark variables, respectively. You can run the following two cells to make sure that the Kernel is active.

**Do not insert any additional cells than the ones that are provided for assignment submission. You may add cells as you work through the notebook, but you need to delete extra cells and keep the initial structure.**

### Note on time it takes to complete the model training

When we designed the assignment, it was thoroughly tested and the training time took no more than 30-40 mins with the provided configuration on the entire dataset. In previous courses, several students told us that the training is taking too long, more than that. There is no direct way to troubleshoot. That is one of the unfortunate realities with working with these tools.

If you do run into problems, you have two options:

- Try adding more core or task nodes to the cluster, and restart your notebook and try again.
- Your other option is  to take a 10% sample of the "encoded_dataset" before you split into 90% training and 10% testing. 

### Some suggestions you may want to follow:

* You may consider saving intermediate datasets (i.e. training and testing) when you first create them. To save intermediate datasets, save them as **parquet** files in your own S3 bucket. To save, use the following code: `df.write.parquet("s3://[[your-s3-bucket]]/data_location/", mode="overwrite")`.
* You may also want to save a model object in S3 after you train it, especially if training takes a while. To save a model object, use the following code: `model.save("s3://[[your-s3-bucket]]/model_location/")`
* When creating the Machine Learning pipelines, you may want to try it first on a minuscule sample of your training data to make sure the pipelines work as planned. To create a tiny DataFrame, use the `limit` method: `df.limit(100)` (this creates a small DataFrame with the first 100 rows from df.)

In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
spark

## Import necessary libraries

The following cell will load all required libraries and functions for this exercise.

In [3]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.feature import RFormula
import matplotlib.pyplot as plt
import numpy as np
import datetime

## Part 1 - Load the data, convert data types and create new features (5 points)

### Load the data

In the following cell, create an object called `nyctaxi` which loads the **parquet** files from `s3://bigdatateaching/nyctaxi-2013/merged-parquet/`.

This step took approximately 8 seconds during the development of the workbook using the cluster configuration in the assignment. The workbook will have spots were running times during development are written for reference as a comment in a cell. Your run times may vary, but these values will give you an indication.

In [4]:
# ~ 8 seconds
nyctaxi= sqlContext.read.parquet('s3://bigdatateaching/nyctaxi-2013/merged-parquet/')

In the next cell print the schema of `nyctaxi`.

In [5]:
nyctaxi.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total: string (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)



In the next cell, count the number of records for `nyctaxi`.

In [6]:
# ~ 12 seconds
print("Number of records: " + str(nyctaxi.count()))

Number of records: 173185091


In the next cell, show the first 10 records of `nyctaxi` to see what the data looks like.

In [7]:
# ~ 4 seconds
nyctaxi.show(10)

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+-----+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+-----+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|00005007A9F30E289...|132A7AC13C8471488...|      CMT|2013-07-30 

### Convert data types

As you can see from printing the `nyctaxi` schema, all of the fields were loaded as strings. This will not work for modeling purposes. You need to convert some of the fields to other types. In the following cell, create a new DataFrame called `nyctaxi_converted` and use the `withColumn` method to do the following conversions:

 Field Name | Type
------------|-----
trip_time_in_seconds | integer
trip_distance | float
pickup_latitude | float
pickup_longitude | float
dropoff_latitude | float
dropoff_longitude | float
fare_amount | float
surcharge | float
mta_tax | float
tip_amount | float
tolls_amount | float
total | float
pickup_datetime | timestamp
dropoff_datetime | timestamp




In [8]:
nyctaxi_converted= nyctaxi.withColumn("trip_time_in_secs", nyctaxi.trip_time_in_secs.cast('Integer'))\
                .withColumn("trip_distance", nyctaxi.trip_distance.cast('float'))\
                .withColumn("pickup_latitude", nyctaxi.pickup_latitude.cast('float'))\
                .withColumn("pickup_longitude", nyctaxi.pickup_longitude.cast('float'))\
                .withColumn("dropoff_latitude", nyctaxi.dropoff_latitude.cast('float'))\
                .withColumn("dropoff_longitude", nyctaxi.dropoff_longitude.cast('float'))\
                .withColumn("fare_amount", nyctaxi.fare_amount.cast('float'))\
                .withColumn("surcharge", nyctaxi.surcharge.cast('float'))\
                .withColumn("mta_tax", nyctaxi.mta_tax.cast('float'))\
                .withColumn("tip_amount", nyctaxi.tip_amount.cast('float'))\
                .withColumn("tolls_amount", nyctaxi.tolls_amount.cast('float'))\
                .withColumn("total", nyctaxi.total.cast('float'))\
                .withColumn("pickup_datetime", nyctaxi.pickup_datetime.cast('timestamp'))\
                .withColumn("dropoff_datetime", nyctaxi.dropoff_datetime.cast('timestamp'))\
                .withColumn("passenger_count", nyctaxi.passenger_count.cast('Integer'))

# drop_list= ['vendor_id','medallion','hack_license','payment_type','rate_code','store_and_fwd_flag',
#             'passenger_count','trip_time_in_secs']
# nyctaxi_converted= nyctaxi.select([column for column in nyctaxi_converted.columns if column not in drop_list])

In [9]:
nyctaxi_converted.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)



### Add new features

You will add a few new fields with data derived from current fields as new features.

In the following cell, add the following columns to `nyctaxi_converted` using the `withColumn` method:

* A column called `pickup_hour` with the hour from `pickup_datetime`. This provides an integer from 0 to 23.
* A column called `pickup_week` with the week of the year from `pickup_datetime`. This provides an integer from 1 to 53.
* A column called `weekday` with the name of the day of the week ffrom `pickup_datetime`, in long format.
* A column called `tipped` which is an indicator of wether or not there was a tip. If the tip is 0, then it's 0, otherwise 1.

In [10]:
# nyctaxi_converted.select('pickup_datetime').show(5)
def valueToCategory(value):
    if   value == 0: return 0
    else: return 1
udfValueToCategory = udf(valueToCategory, IntegerType())

nyctaxi_converted=nyctaxi_converted.withColumn('pickup_hour', hour(col("pickup_datetime")))\
                 .withColumn("pickup_week",weekofyear(col("pickup_datetime")))\
                 .withColumn("weekday", date_format('pickup_datetime', 'E').alias('dow_string'))\
                 .withColumn("tipped", udfValueToCategory("tip_amount"))

For the creation of the next feature, you will use a SparkSQL statement. Therefore, you need to register your DataFrame. In the next cell, register the `nyctaxi_converted` DataFrame as `nyctaxi_converted_tbl.`

In [11]:
nyctaxi_converted.createOrReplaceTempView("nyctaxi_converted_tbl")

In the next cell, you will add a new column called `time_bins` that takes the value of `pickup_hour` and buckets it according to the following rules. You must assign the SQL statement back to `nyctaxi_converted`:

* If the value of the pickup hour is at-or-before 6am, or at-or-after 8pm, then the value is "night"
* If the value of the pickup hour is between 7am and 10am (inclusive), then the value is "am_rush"
* If the value of the pickup hour is between 11am and 3pm (inclusive), then the value is "afternoon"
* If the value of the pickup hour is between 4pm and 7pm (inclusive), then the value is "pm_rush"


In [12]:
def timebins(time):
    if time<=6 and time>=20: return "night"
    elif time>7 and time<=10: return "am_rush"
    elif time>11 and time<=13: return "afternoon"
    elif time>16 and time<=19: return "pm_rush"
    else: return "non_peak"
    
udftimebins = udf(timebins, StringType())
nyctaxi_converted= nyctaxi_converted.withColumn('time_bins',udftimebins('pickup_hour') )

In the next cell, print the schema of your `nyctaxi_converted` DataFrame and make sure it compares to the results below. Field order is not important.

```
root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_week: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- tipped: integer (nullable = false)
 |-- time_bins: string (nullable = true)
```

In [13]:
nyctaxi_converted.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_week: i

In the next cell, show the first 10 rows of `nyctaxi_converted` to see if your new fields are correct.

In [14]:
# ~ 12 seconds
nyctaxi_converted.show(10)

+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+-----+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+-----------+-----------+-------+------+---------+
|           medallion|        hack_license|vendor_id|    pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total|rate_code|store_and_fwd_flag|   dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|pickup_hour|pickup_week|weekday|tipped|time_bins|
+--------------------+--------------------+---------+-------------------+------------+-----------+---------+-------+----------+------------+-----+---------+------------------+-------------------+---------------+-----------------+-------------+----------------+---------------+--

## Part 2 - Drop unused variables and filter data for clean trips

In the following cell, create a new DataFrame called `valid_trips` with the following structure. You are welcome to use DataFrame methods or SparkSQL for this.

* Keep only these fields: time_bins, tipped, weekday, pickup_week, pickup_hour, trip_distance, trip_time_in_secs, passenger_count, rate_code, total, tip_amount, fare_amount, payment_type, vendor_id
* Filter records on the following criteria:
  * Passenger count is greater than 0 and less than 8
  * Payment type is cash (CSH) or credit card (CRD)
  * Tip amount is 0 or more, but less than 30
  * Fare amount is between \\$1 or more, and less than \\$150
  * Trip distance is less than 100 miles, but more than 0
  * The time of the trip is 30 seconds or more, and less than 2 hours

In [15]:
keep_columns= ['time_bins','tipped','weekday','pickup_week','pickup_hour','trip_distance','trip_time_in_secs',
               'passenger_count','rate_code','total','tip_amount','fare_amount','payment_type','vendor_id']

valid_trips= nyctaxi_converted.select([column for column in nyctaxi_converted.columns if column in keep_columns])
valid_trips= valid_trips.filter((valid_trips.passenger_count >0) & (valid_trips.passenger_count<8))\
                        .filter((valid_trips.payment_type == 'CSH') | (valid_trips.payment_type== 'CRD'))\
                        .filter((valid_trips.tip_amount >= 0) & (valid_trips.tip_amount < 30))\
                        .filter((valid_trips.fare_amount>= 1) & (valid_trips.fare_amount< 150))\
                        .filter((valid_trips.trip_distance> 0) & (valid_trips.trip_distance< 100))\
                        .filter((valid_trips.trip_time_in_secs >= 30) & (valid_trips.trip_time_in_secs< 7200))

In the following cell, print the schema of the `valid_trips` DataFrame and compare to this:

```
root
 |-- vendor_id: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_week: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- tipped: integer (nullable = false)
 |-- time_bins: string (nullable = true)
```

In [16]:
valid_trips.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_week: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- tipped: integer (nullable = true)
 |-- time_bins: string (nullable = true)



In the following cell, count the number of records for `valid_trips`.

In [17]:
# ~ 48 seconds
print("Number of records: " + str(valid_trips.count()))

Number of records: 171182914


In the following cell, calculate the number of records that were dropped with the filter applied.

In [18]:
print("Number of records dropped: " + str(nyctaxi.count()- valid_trips.count()))

Number of records dropped: 2002177


## Part 3 - Feature transformation and data preparation for modeling

In this section, you will convert the categorical variables `vendor_id`, `rate_code`, `payment_type`, and `time_bins` to features that can be used in the model. You need to use the `StringIndexer` function on each of the categorical variables to convert the raw text into indices, and then for each set of indices you need to use the `OneHotEncoder` function to convert the index to a vector of dummy variables.

In the next 8 cells, create four `StringIndexer` objects named `si_1` through `si_4`, and four `OneHotEncoder` objects named `en_1` through `en_4` for each of the categorical variables. Each `en_` must use as input, the output from the `si_`. The four output columns for `en_` must be named `vendor_vec`, `rate_vec`, `payment_vec`, `time_bins_vec`.

Make sure that you use the parameter `dropLast=False` for `OneHotEncoder`, and that the `en_` and `si_` of the same number are used on the same variable.

We talked about transformers and estimators in class. Refer to the book for additional information. As a brief summary:
* Transformers: take data in, and produce new data
* Estimators: take data in and produce a transformer

In [19]:
si_1 = StringIndexer(inputCol= 'vendor_id', outputCol= 'vendor_index' )

In [20]:
en_1 = OneHotEncoder(dropLast=False, inputCol = 'vendor_index', outputCol= 'vendor_vec')

In [21]:
si_2 = StringIndexer(inputCol= 'rate_code', outputCol= 'rate_in' )

In [22]:
en_2 = OneHotEncoder(dropLast=False, inputCol = 'rate_in', outputCol= 'rate_vec')

In [23]:
si_3 = StringIndexer(inputCol= 'payment_type', outputCol= 'payment_index' )

In [24]:
en_3 = OneHotEncoder(dropLast=False, inputCol = 'payment_index', outputCol= 'payment_vec')

In [25]:
si_4 = StringIndexer(inputCol= 'time_bins', outputCol= 'time_bins_index' )

In [26]:
en_4 = OneHotEncoder(dropLast=False, inputCol = 'time_bins_index', outputCol= 'time_bins_vec')

In the following cell, build a pipeline called `encoded_final` where you will run the stages in the following order: si_1, en_1, si_2, en_2, etc. You need to run a fit method and a transform method on the `vaild_trips` DataFrame in order to get the desired results.

In [27]:
# ~ 8 min
pipeline = Pipeline(stages= [si_1, en_1, si_2, en_2, si_3, en_3, si_4, en_4])
model1= pipeline.fit(valid_trips)

encoded_final= model1.transform(valid_trips)

In the following cell, show the first 10 rows of `encoded_final`.

In [109]:
encoded_final.show(10)

+---------+------------+-----------+----------+-----+---------+---------------+-----------------+-------------+-----------+-----------+-------+------+---------+------------+-------------+-------+--------------+-------------+-------------+---------------+-------------+
|vendor_id|payment_type|fare_amount|tip_amount|total|rate_code|passenger_count|trip_time_in_secs|trip_distance|pickup_hour|pickup_week|weekday|tipped|time_bins|vendor_index|   vendor_vec|rate_in|      rate_vec|payment_index|  payment_vec|time_bins_index|time_bins_vec|
+---------+------------+-----------+----------+-----+---------+---------------+-----------------+-------------+-----------+-----------+-------+------+---------+------------+-------------+-------+--------------+-------------+-------------+---------------+-------------+
|      CMT|         CRD|       10.5|       3.0| 14.5|        1|              1|              679|          2.5|         22|         31|    Tue|     1| non_peak|         1.0|(2,[1],[1.0])|    0.

In the following cell, split `encoded_final` into `train` and `test` using 90% train, 10% test and a seed of 12345. **You must use the specified seed for reproducibility.**

In [81]:
train, test = encoded_final.randomSplit([0.9, 0.1], seed=12345)

In the next cell, cache the `train` DataFrame.

In [29]:
train.cache()

DataFrame[vendor_id: string, payment_type: string, fare_amount: float, tip_amount: float, total: float, rate_code: string, passenger_count: int, trip_time_in_secs: int, trip_distance: float, pickup_hour: int, pickup_week: int, weekday: string, tipped: int, time_bins: string, vendor_index: double, vendor_vec: vector, rate_in: double, rate_vec: vector, payment_index: double, payment_vec: vector, time_bins_index: double, time_bins_vec: vector]

In the next cell, cache the `test` DataFrame.

In [30]:
test.cache()

DataFrame[vendor_id: string, payment_type: string, fare_amount: float, tip_amount: float, total: float, rate_code: string, passenger_count: int, trip_time_in_secs: int, trip_distance: float, pickup_hour: int, pickup_week: int, weekday: string, tipped: int, time_bins: string, vendor_index: double, vendor_vec: vector, rate_in: double, rate_vec: vector, payment_index: double, payment_vec: vector, time_bins_index: double, time_bins_vec: vector]

Count the number of records in the `train` DataFrame.

In [32]:
# ~ 10 min - this takes time because you are actually loading the train dataset into memory as you count. 
train.printSchema()

root
 |-- vendor_id: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- total: float (nullable = true)
 |-- rate_code: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- pickup_week: integer (nullable = true)
 |-- weekday: string (nullable = true)
 |-- tipped: integer (nullable = true)
 |-- time_bins: string (nullable = true)
 |-- vendor_index: double (nullable = false)
 |-- vendor_vec: vector (nullable = true)
 |-- rate_in: double (nullable = false)
 |-- rate_vec: vector (nullable = true)
 |-- payment_index: double (nullable = false)
 |-- payment_vec: vector (nullable = true)
 |-- time_bins_index: double (nullable = false)
 |-- time_bins_vec: vector (nullable = true)



## Part 4 - Build a Logistic Regression Model to predict tipping

In this section, you will train a Logistic Regression model to predict wether or not there was a tip for each ride using the training data created in the previous section. You will build pipelines using both transformers and estimators. 

In the next cell, create a `LogisticRegression` estimator called `log_reg` with the following parameters: `maxIter = 10, regParam = 0.3, eslaticNetParam = 0.8`.

In [33]:
log_reg = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

The Logistic Regression model you will be fitting is predicting `tipped` as a function of:
* Pickup hour
* Passenger count
* Trip time
* Trip distance
* Fare amount
* Vendor id
* Payment type
* Rate code
* Time bins

Remember, you can't use the raw categorical data, you need to use something else for the categorical variables.

In the next cell, create an object called `class_formula` and build a formula for the regression based on the previous instructions.


In [34]:
class_formula= VectorAssembler(inputCols= ['pickup_hour','passenger_count','trip_time_in_secs','trip_distance','fare_amount','vendor_vec',
                'payment_vec', 'rate_vec','time_bins_vec'], outputCol= "features")

In the next cell, create a PipelineModel object called model, with the stages of `class_formula` and `log_reg`, in that order. Run the fit method on the `train` DataFrame. This creates a transformer. 

In [35]:
train1= train.limit(100)
train1= train1.selectExpr("tipped as label", "pickup_hour as pickup_hour","passenger_count as passenger_count",
                         "trip_time_in_secs as trip_time_in_secs", "trip_distance as trip_distance","fare_amount as fare_amount",
                         "vendor_vec as vendor_vec","payment_vec as payment_vec", "rate_vec as rate_vec", "time_bins_vec as time_bins_vec"  )
train1.printSchema()
# train1= train1.select('pickup_hour','passenger_count','trip_time_in_secs','trip_distance','fare_amount','vendor_vec','payment_vec', 'rate_vec','time_bins_vec','label')
model= Pipeline(stages= [class_formula, log_reg]) 
m1= model.fit(train1)

root
 |-- label: integer (nullable = true)
 |-- pickup_hour: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- vendor_vec: vector (nullable = true)
 |-- payment_vec: vector (nullable = true)
 |-- rate_vec: vector (nullable = true)
 |-- time_bins_vec: vector (nullable = true)



In the next cell, create a `predictions` DataFrame by taking the model produced in the previous cell and running the `transform` method on the `test` data.

In [36]:
test = test.limit(50)
test= test.selectExpr("pickup_hour as pickup_hour","passenger_count as passenger_count",
                      "trip_time_in_secs as trip_time_in_secs", "trip_distance as trip_distance",
                      "fare_amount as fare_amount","vendor_vec as vendor_vec","payment_vec as payment_vec", 
                      "rate_vec as rate_vec", "time_bins_vec as time_bins_vec" ,"tipped as label")
test.printSchema() 
predictions= m1.transform(test)
predictions.printSchema()

root
 |-- pickup_hour: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- vendor_vec: vector (nullable = true)
 |-- payment_vec: vector (nullable = true)
 |-- rate_vec: vector (nullable = true)
 |-- time_bins_vec: vector (nullable = true)
 |-- label: integer (nullable = true)

root
 |-- pickup_hour: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- vendor_vec: vector (nullable = true)
 |-- payment_vec: vector (nullable = true)
 |-- rate_vec: vector (nullable = true)
 |-- time_bins_vec: vector (nullable = true)
 |-- label: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullab

In the next cell, create an object called `predictions_and_labels` which takes the `label` and `prediction` columns from the `predictions` DataFrame and converts it to an RDD.

In [91]:
predictions= predictions.withColumn("pred", predictions.prediction.cast('float'))\
                        .withColumn("lab", predictions.label.cast('float'))
pred_rd= predictions.select("pred", "lab")

def getpredictions_and_labels(x):
    
    pred_rd= x.asDict()
    ret_tuple= (pred_rd['pred'],pred_rd['lab'])
    return ret_tuple

predictions_and_labels= predictions.rdd.map(getpredictions_and_labels)

In the next cell, create an evaluator called metrics, and use the `BinaryClassificationMetrics` evaluator on the `predictions_and_labels` object.

In [93]:
metrics= BinaryClassificationMetrics(predictions_and_labels)

In this last step, the code is there for you to run. The next cell calculates the Area Under the Curve for the ROC curve generated by the model.

In [94]:
# ~13 min
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under ROC = 0.5


What do you think of the result of the Area under ROC?

In [ ]:
## type answer here. Do not evaluate this cell.
Each point on the ROC curve represents a sensitivity/specificity pair corresponding to a particular decision threshold.
The area under the ROC curve ( AUC ) is a measure of how well a parameter can distinguish between two diagnostic groups
(diseased/normal).
An area ROC equal to 0.5 (i.e. coinciding with the diagonal) indicates a random classification model. 

## Extra Credit (2 points)

For extra credit, take the `label` and `probability` columns from the `predictions` DataFrame and create a Pandas DataFrame (local to the master node), and plot a ROC curve.

In the following cell, make a local Pandas DataFrame from `predictions`. **Hint:** you may want to randomly sample about 20% of the results, not use the entire `predictions`. 

In the following cell, use `matplotlib` and `scikit-learn` to plot the ROC curve.

## (For Reference) Load a saved pipeline model and evaluate it on a data set

If you did save a model as suggested in the beginning of the workbook, you can load it again.

In [ ]:
from pyspark.ml import PipelineModel

saved_model = PipelineModel.load("s3://[[your-s3-bucket]]/model_location/")
test = spark.read.parquet("s3://[[your-s3-bucket]]/data_location/")

predictions = saved_model.transform(test)
prediction_and_labels = predictions.select("label","prediction").rdd

metrics = BinaryClassificationMetrics(prediction_and_labels)
print("Area under ROC = %s" % metrics.areaUnderROC)